## Imports 

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
from statsmodels.distributions.empirical_distribution import ECDF
import os
import glob
import copy

## Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',', index_col=0) for file in sorted(glob.glob('../preprocessed_datasets' + "/*."+'csv'))]
cohorts = [file.strip(".csv") for file in sorted(os.listdir('../preprocessed_datasets'))]

In [3]:
# reduce to BL visit and AD participants only
all_cohorts = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts[name] = df.loc[(df["Visit"]==1) & (df["Diagnosis"].astype(str)=='AD')]
    
#all_cohorts_ctl = copy.deepcopy(all_cohorts)
all_cohorts_ctl = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts_ctl[name] = df.loc[(df["Visit"]==1) & (df["Diagnosis"].astype(str)=='CU')]   

## Functions to perform essential calculations 

In [4]:
def cat_stat_df(dfs, result):
    """Counting different categories, calculate the % of categorical features, store results in a df"""
    
    categorical = {'APOE4': [2.0, 1.0], 'Sex': ['Female'], 'Diagnosis': ['CU', 'MCI', 'AD']}
    column_cat = ['Sex', 'Diagnosis', 'APOE4']

    for cohort in dfs:
        
        if dfs[cohort].empty==False:
            calc_dict = dict()
            df = dfs[cohort]

            for col in column_cat:
                ca = Counter(df[col].dropna())
                calc_dict[col] = ca

            cohort_df = pd.DataFrame(calc_dict).transpose()
            cohort_df = cohort_df.dropna(how='all')
            cohort_df.loc[cohort] = cohort_df.sum()
           
            for i in categorical:
                
                if i == 'Diagnosis':
                    
                    if i in cohort_df.index: 
                        result.loc[cohort, categorical[i]] = cohort_df.loc[cohort, cohort_df.loc[i].notna()].astype(int)
                        result.loc[cohort, categorical[i]] = result.loc[cohort, categorical[i]].replace({np.nan: 0})
                        result.loc[cohort, 'n'] = int(sum(cohort_df.loc[cohort, cohort_df.loc[i].notna()]))
                    
                    else:
                        result.loc[cohort, i] = np.nan
                        result.loc[cohort, 'n'] = int(len(dfs[cohort].index))
                
                elif i == 'APOE4':
                    
                    if 'APOE4' in list(cohort_df.index.astype(str)):
                        
                        if '2.0' not in list(cohort_df.columns.astype(str)) and '2' not in list(cohort_df.columns.astype(str)):
                            cohort_df[2.0] = np.nan
                        
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) / 
                                                     sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    
                    else:
                        result.loc[cohort, i] = np.nan
                
                elif i == 'Sex':
                    
                    if (i in cohort_df.index) & ("Female" in cohort_df.columns):
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) 
                                                      / sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    else:
                        result.loc[cohort, i] = 0
                        
        result.loc[cohort, 'Total'] = int(len(dfs[cohort].index))
    
                    
    result.rename(columns={"Sex": "Female %", "APOE4": "APOE4 %"}, inplace=True)
              
    return result

In [5]:
def num_quantiles(dfs, dfs_ctl, result):
    """Calculating std and mean and evalute quantiles under the ECDF for all cohorts"""
    
    column_num = ['Age', 'CDR', 'Education', 'MMSE', 'CDRSB', 'Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    non_int_cols=["CDR", "CDRSB"]
    biomarker = ['Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    
    for df, ctl_df in zip(dfs, dfs_ctl):
        
        dfn = dfs[df]
        dfn_ctl = dfs_ctl[ctl_df]
        calc_dict = dict()
        calc_dict_ctl = dict()
        
        for col in column_num:
            quants = []
            
            if (dfn.empty == False) & (col in dfn.columns)==True:
                
                if len(dfn.index.unique()) > 2:

                    # return nan if no data
                    if pd.isnull(dfn[col].quantile()):
                        calc_dict[col] = np.nan
                        continue

                    if col in non_int_cols:

                        for i in [.25, .5, .75]:
                            quants.append(round(dfn[col].quantile(i), 1))
                    else:

                        for i in [.25, .5, .75]:
                            quants.append(int(round(dfn[col].quantile(i), 0)))

                    # create and save string to return
                    calc_dict[col] =  str(quants[0]) + ', ' + str(quants[1]) + ', ' + str(quants[2])
                
                elif len(dfn.index.unique()) == 2:
                    
                    if col == 'Age':
                        quants = (list(dfn.iloc[0:][col].values))
                        calc_dict[col] = str(int(quants[0])) +  ', ' + str(int(quants[1]))
                     
                    else:
                        quants = (list(dfn.iloc[0:][col].values))
                        calc_dict[col] = str(round(quants[0], 1)) +  ', ' + str(round(quants[1], 1))
                
                else:
                    
                    if col == 'Age':
                        calc_dict[col] = int(dfn.iloc[0][col]) 
                    
                    else:
                        calc_dict[col] = round(dfn.iloc[0][col], 1)
                    

        for col in biomarker:
            if len(dfn_ctl.index.unique()) > 2:

                if (dfn_ctl.empty == False) & (col in dfn.columns)==True:

                    quants = []
                    ctl_dat = dfn_ctl[col].dropna()

                    # return nan if no data or no control distribution to compare to
                    if (pd.isnull(dfn[col].quantile())) or not (ctl_dat.any()):
                        calc_dict_ctl[col] = np.nan
                        continue

                    ecdf = ECDF(ctl_dat)

                    for i in [.25, .5, .75]:
                        value = int(round(ecdf(dfn[col].quantile(i)) * 100))
                        quants.append(value)

                    # create and save string to return
                    calc_dict_ctl[col] =  str(quants[0]) + ', ' + str(quants[1]) + ', ' + str(quants[2])
                
                else:
                    calc_dict_ctl[col] = np.nan
                

        for clin, bio in zip([calc_dict], [calc_dict_ctl]):
        
            for marker in biomarker:
                
                if (marker in clin) & (marker in bio): 

                    if pd.notnull(clin[marker]):
                        clin[marker] += " (" + str(bio[marker]) + ")"
                        
                else:
                    continue

        df_quan = pd.DataFrame(calc_dict, index=[df])
        
        for col in df_quan.columns:
            result.loc[df, col] = df_quan.loc[df, col]
            
    result.rename(columns={"Ttau": "tTau", "Ptau": "pTau"}, inplace=True)
        
    return result

## Make an empty dataframe to fill in with the results

In [6]:
results = pd.DataFrame(index = all_cohorts.keys(), columns = [col for col in all_cohorts['AIBL'].columns])
results.index.name = 'Name of Dataset'

for i in ['CU', 'MCI', 'AD', 'Total']:
    results[i] = np.nan
    
results.drop(columns=['Diagnosis', 'Visit'], inplace=True)

results = cat_stat_df(all_cohorts, results)
results = num_quantiles(all_cohorts, all_cohorts_ctl, results)

## Final table 

In [7]:
# sort columns
results = results[["n", "Total", "CU", "MCI", "AD", "Female %", "Age", "Education", "APOE4 %", 
                   "MMSE", "CDR", "CDRSB", "Hippocampus", "A-beta", "tTau", "pTau"]]

In [8]:
results

,n,Total,CU,MCI,AD,Female %,Age,Education,APOE4 %,MMSE,CDR,CDRSB,Hippocampus,A-beta,tTau,pTau
Name of Dataset,,,,,,,,,,,,,,,,
A4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABVIB,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,389.0,389.0,0.0,0.0,389.0,43.7,"71, 75, 80","13, 16, 18",66.7,"21, 23, 25","0.5, 1.0, 1.0","3.0, 4.5, 5.0","4990, 5640, 6456 (0, 3, 16)","470, 594, 752 (5, 12, 21)","268, 338, 440 (71, 85, 96)","26, 33, 44 (73, 89, 96)"
AIBL,181.0,181.0,0.0,0.0,181.0,63.5,"73, 80, 85","10, 12, 13",61.9,"15, 20, 23","0.5, 1.0, 1.0","3.5, 5.0, 7.0","2, 2, 3 (2, 6, 16)",NaN,NaN,NaN
ANM,512.0,512.0,0.0,0.0,512.0,62.9,"74, 79, 83","6, 9, 12",54.3,"16, 21, 25","0.5, 1.0, 1.0","4.0, 6.0, 9.0","4694, 5199, 5828 (1, 2, 9)",NaN,NaN,NaN
ARWIBO,281.0,281.0,0.0,0.0,281.0,66.9,"69, 74, 80","5, 5, 8",41.2,"18, 21, 24","1.0, 1.0, 1.0",NaN,"4862, 5441, 6058 (0, 1, 4)","273, 355, 421 (0, 0, 0)","453, 604, 828 (0, 100, 100)","62, 83, 100 (0, 0, 100)"
DOD-ADNI,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EDSD,151.0,151.0,0.0,0.0,151.0,57.6,"68, 73, 79","8, 11, 13",57.1,"19, 22, 25",NaN,NaN,"4842, 5640, 6336 (1, 2, 5)","634, 634, 634 (nan)","1150, 1150, 1150 (nan)","156, 156, 156 (nan)"
EMIF,201.0,201.0,0.0,0.0,201.0,46.8,"63, 71, 77","8, 10, 13",NaN,"18, 23, 25","0.5, 1.0, 1.0",NaN,"5511, 6224, 6957 (0, 3, 22)","275, 394, 502 (2, 11, 31)","150, 341, 705 (33, 85, 98)","39, 59, 91 (47, 82, 97)"


### Outputs

In [9]:
results[["Female %", "Age", "Education", "APOE4 %", "MMSE", "CDR", "CDRSB", "Hippocampus", 
        "A-beta", "tTau", "pTau"]].to_csv("../adata_resources/AD_summary_stats.csv")

In [10]:
print("N all cohorts: ", results["n"].sum())

N all cohorts:  14129.0
